In [ ]:
'''
Source: https://www.youtube.com/watch?list=PLQVvvaa0QuDf2JswnfiGkliBInZnIC4HL&v=eObouMO2qSE
Author: Shivam Dhole
Student No: 741507

Description:
This code loads six classifiersusing pickle. These clsssifiers are pre trained and stored in text files.
The code loades these trained classifiers and uses it to find sentiment of a given string.
This code is run on tweets stored in CouchDB.
The tweets are filtered using an attribute called 'python'.
All the tweets who have this attribute false, sentiment analysis using NLTK is done on them.
The second sentiment analysis is performed using Textblob.
The same text goes through the Textblob function and returns polarity of the tweet.
The sentiment value is calculated using both methods and the document is updated with it.
'''

import nltk
import random
import couchdb
import datetime
from textblob import TextBlob
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize



class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf


documents_f = open("documents.pickle", "rb")
documents = pickle.load(documents_f)
documents_f.close()




word_features5k_f = open("word_features5k.pickle", "rb")
word_features = pickle.load(word_features5k_f)
word_features5k_f.close()


def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features



featuresets_f = open("word_features5k.pickle", "rb")
featuresets = pickle.load(featuresets_f)
featuresets_f.close()

random.shuffle(featuresets)
print(len(featuresets))

testing_set = featuresets[10000:]
training_set = featuresets[:10000]



open_file = open("originalnaivebayes5k.pickle", "rb")
classifier = pickle.load(open_file)
open_file.close()


open_file = open("MNB_classifier5k.pickle", "rb")
MNB_classifier = pickle.load(open_file)
open_file.close()



open_file = open("BernoulliNB_classifier5k.pickle", "rb")
BernoulliNB_classifier = pickle.load(open_file)
open_file.close()


open_file = open("LogisticRegression_classifier5k.pickle", "rb")
LogisticRegression_classifier = pickle.load(open_file)
open_file.close()


open_file = open("LinearSVC_classifier5k.pickle", "rb")
LinearSVC_classifier = pickle.load(open_file)
open_file.close()


open_file = open("SGDC_classifier5k.pickle", "rb")
SGDC_classifier = pickle.load(open_file)
open_file.close()




voted_classifier = VoteClassifier(
                                  classifier,
                                  LinearSVC_classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LogisticRegression_classifier)




def sentiment(text):
    feats = find_features(text)
    return voted_classifier.classify(feats),voted_classifier.confidence(feats)

# Getting document and performing analysis starts here
couch = couchdb.Server('http://xxx.xxx.xxx.xxx:5984/')

db = couch['newtweets']
count = 0
for docid in db.view('shivam/pythonfalse'):
    i = docid['id']
    doc = db.get(i)
    text = doc["text"]
    testimonial = TextBlob(text)
    s = sentiment(text)
    pol = testimonial.polarity
    sub = testimonial.subjectivity
    doc["NLTKsentiment"] = s[0]
    doc["textblobpolarity"] = pol
    doc["textblobsubjectivity"] = sub
    doc["python"] = True
    doc = db.save(doc)
    count = count+1
    if count%10000 == 0: # To track progress of code
        print(count,datetime.datetime.now())
print("Finish")

5000
